**Библиотеки**

In [ ]:
import os
import sys
import logging
import numpy as np
import sympy as sp
import jax.numpy as jnp
from sympy import Eq, Rel
from jax import jacfwd
from datetime import datetime
from pathlib import Path
from typing import Callable

**Параметры**

In [ ]:
MAX_ITER = 1000
INPUT_FILE_PATH = None # "input.txt" - Использовать файл для ввода
LOG_IN_FILE = True # Сохранять логи в файл

**Функции**

In [ ]:
def setup_logger(log_in_file: bool):
    logger = logging.getLogger("my_logger")
    logger.setLevel(logging.INFO)
    logger.handlers.clear()
    logger.propagate = False

    console_handler = logging.StreamHandler(sys.stdout)
    console_handler.setFormatter(logging.Formatter("%(message)s"))
    logger.addHandler(console_handler)

    if log_in_file:
        logs_dir = Path("logs")
        logs_dir.mkdir(exist_ok=True)

        timestamp = datetime.now().strftime("%Y-%m-%d %H-%M-%S")
        file_path = logs_dir / f"log_{timestamp}.txt"
        file_handler = logging.FileHandler(file_path, encoding="utf-8")
        file_handler.setFormatter(logging.Formatter("%(message)s"))
        logger.addHandler(file_handler)

    return logger

In [ ]:
def get_constraint_expression(constraint_string: str):
    constraint_string = constraint_string.strip()

    try:
        if "=" in constraint_string and ">=" not in constraint_string and "<=" not in constraint_string and "!=" not in constraint_string:
            constraint_string = constraint_string.replace("==", "=")
            left_part, right_part = constraint_string.split("=")
            lhs = sp.sympify(left_part)
            rhs = sp.sympify(right_part)
            constraint_expression = sp.Eq(lhs, rhs)
        else:
            constraint_expression = sp.sympify(constraint_string)

        while isinstance(constraint_expression, sp.Not) and isinstance(constraint_expression.args[0], sp.Eq):
            constraint_expression = constraint_expression.args[0]

        if isinstance(constraint_expression, Eq):
            return "equality", constraint_expression.lhs - constraint_expression.rhs
        elif isinstance(constraint_expression, Rel):
            return "inequality", constraint_expression.lhs - constraint_expression.rhs
        else:
            return None, None
    except Exception as e:
        logger.error(f"Ошибка при обработке выражения '{constraint_string}': {e}")
        return None, None

In [ ]:
def read_params_from_file(file_path: Path):
    x, s, lambdas, mu, epsilon = None, None, None, None, None
    equalities_expressions, inequalities_expressions = [], []

    target_function_expression = None
    constraints_started = False

    with open(file_path, "r", encoding="utf-8") as file:
        for line in file:
            line = line.strip()
            if not line or line.startswith("#"):
                continue

            if line.lower().startswith("constraints"):
                constraints_started = True
                continue

            if constraints_started:
                constraint_type, constraint_expression = get_constraint_expression(line)
                if constraint_type == "equality":
                    equalities_expressions.append(constraint_expression)
                elif constraint_type == "inequality":
                    inequalities_expressions.append(constraint_expression)
                continue

            if ":" not in line:
                continue
            key, value = line.split(":", 1)
            key = key.strip()
            value = value.strip()

            if key == "target_function":
                target_function_expression = sp.sympify(value)
                x_symbols = sorted(target_function_expression.free_symbols, key=lambda s: s.name)
                target_function = sp.lambdify(x_symbols, target_function_expression)
            elif key == "x":
                x = np.array([float(i) for i in value.split()])
            elif key == "s":
                s = np.array([float(i) for i in value.split()])
            elif key == "lambdas":
                lambdas = np.array([float(i) for i in value.split()])
            elif key == "mu":
                mu = float(value)
            elif key in ("epsilon", "eps"):
                epsilon = float(value)

    equalities_count = len(equalities_expressions)
    inequalities_count = len(inequalities_expressions)

    s_symbols = sp.symbols(f"s1:{inequalities_count + 1}")
    lambda_symbols = sp.symbols(f'λ1:{equalities_count + 1}')
    v_symbols = sp.symbols(f'ν1:{inequalities_count + 1}')

    return x, x_symbols, s, s_symbols, lambdas, lambda_symbols, v_symbols, mu, epsilon, target_function, target_function_expression, equalities_expressions, inequalities_expressions

In [ ]:
def read_params_from_input():
    target_function_string = input("Введите целевую функцию: ")

    target_function_expression = sp.sympify(target_function_string)
    target_function_variables = sorted(target_function_expression.free_symbols, key=lambda s: s.name)
    target_function = sp.lambdify(target_function_variables, target_function_expression)
    os.system('cls')

    equalities_expressions = []
    inequalities_expressions = []

    while True:
      constraint_string = input("Введите ограничения (для окончания ввода отправьте пустую строку): ")

      if not constraint_string:
        break

      constraint_type, constraint_expression = get_constraint_expression(constraint_string)

      if not constraint_type:
        pass

      if constraint_type == "equality":
        equalities_expressions.append(constraint_expression)
      elif constraint_type == "inequality":
        inequalities_expressions.append(constraint_expression)

      os.system('cls')

    equalities_count = len(equalities_expressions)
    inequalities_count = len(inequalities_expressions)

    x_symbols = target_function_variables
    lambda_symbols = sp.symbols(f'λ1:{equalities_count + 1}')
    v_symbols = sp.symbols(f'ν1:{inequalities_count + 1}')
    s_symbols = sp.symbols(f"s1:{inequalities_count + 1}")

    os.system('cls')

    x = []

    for variable in target_function_variables:
      x.append(float(input(f'Введите начальное приближение {variable}: ')))

    x = np.array(x)
    os.system('cls')

    s = []

    for s_symbol in s_symbols:
      while True:
        s_value = float(input(f'Введите начальные значения дополнительных переменной {s_symbol}: '))
        if s_value > 0:
            s.append(s_value)
            break
        else:
            print("Значение должно быть строго положительным (s > 0). Попробуйте снова.")

    s = np.array(s)
    os.system('cls')

    lambdas = []

    for lambda_symbol in lambda_symbols:
      lambdas.append(float(input(f'Введите начальные значения многочлена Лагранжа {lambda_symbol}: ')))
    lambdas = np.array(lambdas)
    os.system('cls')

    mu = float(input('Введите барьерный параметр μ: '))
    os.system('cls')

    epsilon = float(input('Введите допустимую погрешность ε: '))
    os.system('cls')

    return x, x_symbols, s, s_symbols, lambdas, lambda_symbols, v_symbols, mu, epsilon, target_function, target_function_expression, equalities_expressions, inequalities_expressions

In [ ]:
def create_lagrange_function(target_function_expression, equalities_experessions, inequalities_expressions, mu, s_symbols, lambda_symbols, v_symbols):
  L_expression = target_function_expression

  L_expression -= mu * sum(sp.log(s) for s in s_symbols)

  for i, h in enumerate(equalities_experessions):
      L_expression += lambda_symbols[i] * h

  for j, g in enumerate(inequalities_expressions):
      L_expression += v_symbols[j] * (g + s_symbols[j])

  v_subs = {v: mu / s for v, s in zip(v_symbols, s_symbols)}
  L_expression = L_expression.subs(v_subs)

  return L_expression

In [ ]:
def create_KKT_system(lagrange_function, x_symbols, s_symbols, lambda_symbols):
  grad_x = [sp.diff(lagrange_function, x) for x in x_symbols]
  grad_lambda = [sp.diff(lagrange_function, lam) for lam in lambda_symbols]
  grad_s = [sp.diff(lagrange_function, s) for s in s_symbols]

  system_equations_expr = grad_x + grad_lambda + grad_s
  all_symbols = x_symbols + list(lambda_symbols) + list(s_symbols)
  system_equations_func = sp.lambdify(all_symbols, system_equations_expr, modules="jax")

  def system_equations(y):
      return jnp.array(system_equations_func(*y))
  return system_equations

In [ ]:
def interior_point_method(
    x: list[float],
    lambd: list[float],
    s: list[float],
    epsilon: float,
    max_iter: float,
    system_equations: Callable,
) -> list[list[float], float, list[float]]:
    y = jnp.array(np.concatenate([x, lambd, s]))
    x_count = len(x)
    s_count = len(s)

    for i in range(max_iter):
        F = system_equations(y)

        residual = jnp.linalg.norm(F)
        logger.info(f'Невязка: {residual}')

        if (residual < epsilon):
            logger.info(f"Невязка меньше ε: {residual} < {epsilon}\n")
            break
        else:
          logger.info("")

        logger.info(f"Итерация {i + 1}")

        logger.info(f'F(y{i}) = ({", ".join([str(i) for i in F])})')

        J = jacfwd(system_equations)(y)
        logger.info(f'J(y{i}):\n {J}')

        logger.info("")
        delta = jnp.linalg.solve(J, -F)
        logger.info(f'Δ{i + 1} = ({", ".join([str(i) for i in delta])})')

        step = 1.0
        max_backtrack = 20

        while max_backtrack > 0:
          y_candidate = y + step * delta
          s_new = y_candidate[-s_count:]

          if np.all(s_new > 0):
            break

          step *= 0.5
          max_backtrack -= 1

          logger.info(
            f"Переменные s содержат отрицательные элементы. "
            f"Уменьшаем шаг до {step}"
          )

        if max_backtrack == 0:
            logger.warning("Не удалось найти шаг, сохраняющий s > 0. Прерывание.")
            break

        y = y_candidate

        logger.info(f'y{i + 1} = ({", ".join([str(i) for i in y])})')

        F = system_equations(y)
        logger.info(f'F(y{i + 1}) = ({", ".join([str(i) for i in F])})\n')
    else:
        logger.info("Достигнуто максимальное число итераций.")

    return y[:x_count], y[x_count], y[x_count + 1:]

In [ ]:
logger = setup_logger(LOG_IN_FILE)

if not INPUT_FILE_PATH:
    x, x_symbols, s, s_symbols, lambdas, lambda_symbols, v_symbols, mu, epsilon, target_function, target_function_expression, equalities_expressions, inequalities_expressions = read_params_from_input()
else:
    input_file_path = Path(INPUT_FILE_PATH)

    if not input_file_path.exists():
        raise FileNotFoundError(f"Файл \"{input_file_path}\" не существует!")

    x, x_symbols, s, s_symbols, lambdas, lambda_symbols, v_symbols, mu, epsilon, target_function, target_function_expression, equalities_expressions, inequalities_expressions = read_params_from_file(INPUT_FILE_PATH)

lagrange_function = create_lagrange_function(target_function_expression, equalities_expressions, inequalities_expressions, mu, s_symbols, lambda_symbols, v_symbols)
KKT_system = create_KKT_system(lagrange_function, x_symbols, s_symbols, lambda_symbols)

logger.info('Начальные параметры:\n')

logger.info('Начальное приближение x:')
for i in range(len(x)):
    logger.info(f'x{i + 1} = {x[i]}')
logger.info('')

logger.info('Начальные значения дополнительных переменных s:')
for i in range(len(s)):
    logger.info(f's{i + 1} = {s[i]}')

logger.info('Начальные значения многочлена Лагранжа λ:')
for i in range(len(lambdas)):
    logger.info(f'λ{i + 1} = {lambdas[i]}')
logger.info('')

logger.info(f'μ = {mu}')
logger.info(f'ε = {epsilon}\n')

logger.info("Метод Ньютона:\n")

x_res, lambd_res, s_res = interior_point_method(
    x,
    lambdas,
    s,
    epsilon,
    MAX_ITER,
    KKT_system
)

logger.info('Вычисленное оптимальное решение:')

for i in range(len(x_res)):
    logger.info(f'x{i + 1}* = {x_res[i]}')
logger.info(f'λ* = {lambd_res}')
for i in range(len(s_res)):
    logger.info(f's{i + 1}* = {s_res[i]}')

logger.info(f'Значение целевой функции F(x*) = {target_function(*x_res)}')

Введите целевую функцию: x1^2 + x2^2 + x3^2
Введите ограничения (для окончания ввода отправьте пустую строку): x1^2 + x2 - 2 = 0
Введите ограничения (для окончания ввода отправьте пустую строку): x1^2 + x2^2 + x3^2 - 4 <= 0
Введите ограничения (для окончания ввода отправьте пустую строку): -x3 <= 0
Введите ограничения (для окончания ввода отправьте пустую строку): 
Введите начальное приближение x1: 1
Введите начальное приближение x2: 1
Введите начальное приближение x3: 1
Введите начальные значения дополнительных переменной s1: 1
Введите начальные значения дополнительных переменной s2: 1
Введите начальные значения многочлена Лагранжа λ1: 0
Введите барьерный параметр μ: 0.1
Введите допустимую погрешность ε: 0.001
Начальные параметры:

Начальное приближение x:
x1 = 1.0
x2 = 1.0
x3 = 1.0

Начальные значения дополнительных переменных s:
s1 = 1.0
s2 = 1.0
Начальные значения многочлена Лагранжа λ:
λ1 = 0.0

μ = 0.1
ε = 0.001

Метод Ньютона:

Невязка: 3.753664970397949

Итерация 1
F(y0) = (2.2